In [10]:
import numpy as np #numpy
import pandas as pd #pandas
import time

import sklearn
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense# Neural network
import keras.backend as K
from PIL import Image
import ROOT

ModuleNotFoundError: No module named 'PIL'

In [ ]:
tf.__version__

In [ ]:
## define function for extracting data of h5 file
def extract_data(chosen_MM, header, data): ### Fct for extracting the right data and storing it in a list
    chosen_data = []
    chosen_index = header.index(chosen_MM)
    j=0    
    while(j < len(data)):
        chosen_data.append(data[j][chosen_index])
        j+=1

    return np.asarray(chosen_data)
######

In [ ]:
#loading data
import h5py

filename = '/mnt/raid/groups/mm-clustering/data/toydata/ToyCluster1_Chan61_Mult5_MPVCharge1000_Sigma5.h5'
amount_of_data = 10 #first <amount of data> entries extracted from data

with h5py.File(filename, 'r') as f:

    keyword = list(f.keys())[0] #select first keyword - in our case ClusterTree
    
    #extracting headers, names of the histos in ClusterTree
    i=0
    header = [] #list with histo names
    while True:
        search_string = "FIELD_"+str(i)+"_NAME"
        name = f[keyword].attrs.get(search_string)
        if(name == None):
            break
        header.append(name.decode("utf-8")) #list
        i += 1
    #print(header)

    #extracting data
    data = np.asarray(f[keyword][:amount_of_data]).tolist() #extracting as numpy.void, converting in numpy.ndarray and then to list...

    #leaving the with, closing file

#from choosing which data, e.g. MM1X, we get right data from list
print(header)
chosen_MM = "Strips_Processed"
chosen_MM_solution = "Strips_Physical"

#we need numpy arrays as input...
hit_data = extract_data(chosen_MM, header, data)
hit_true = extract_data(chosen_MM_solution, header, data)

In [ ]:
x = hit_data[9].reshape(1,305)
y = hit_data[2].reshape(305,1)
heat = x+y

import seaborn as sns

sns.heatmap(heat)


In [ ]:
x = np.arange(0,305)

plt.plot(x, hit_data[9])
plt.plot(x, hit_data[2], color='black')
plt.show()

In [ ]:
hit_data.shape

In [ ]:
x = np.random.randn(4096)
print(x.shape)
y= np.random.randn(4096)

# Create heatmap
heatmap, xedges, yedges = np.histogram2d(x, y, bins=(64,64))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

# Plot heatmap
plt.clf()
plt.title('Pythonspot.com heatmap example')
plt.ylabel('y')
plt.xlabel('x')
plt.imshow(heatmap, extent=extent)
plt.show()

In [ ]:
target_x_dim = 18
target_shape= target_x_dim,target_x_dim
target_len = 18*18

In [ ]:
# reshape hit_data into square arrays
nulls_len = target_len - hit_true[0].shape[0]

hit_data_true_padded = np.append(hit_true, np.zeros((hit_true.shape[0], nulls_len)), axis=1)

hit_data_true_square = []

for row in hit_data_true_padded:
    row_square = np.reshape(row, target_shape)
    img = Image.fromarray((row_square))
    img = img.resize((18,18), Image.ANTIALIAS)
    hit_data_true_square.append(img)

In [ ]:
# reshape hit_data into square arrays
nulls_len = target_len - hit_data[0].shape[0]

hit_data_padded = np.append(hit_data, np.zeros((hit_data.shape[0], nulls_len)), axis=1)

hit_data_square = []

for row in hit_data_padded:
    row_square = np.reshape(row, target_shape)
    img = Image.fromarray((row_square))
    img = img.resize((18,18), Image.ANTIALIAS)
    hit_data_square.append(img)

In [ ]:

#hit_data = sklearn.preprocessing.minmax_scale(hit_data, feature_range=(0, 1))
#hit_true = sklearn.preprocessing.minmax_scale(hit_true, feature_range=(0, 1))
#hit_true

In [ ]:
# reshape hit_data into square arrays
nulls_len = target_len - hit_data[0].shape[0]

hit_data_padded = np.append(hit_data, np.zeros((hit_data.shape[0], nulls_len)), axis=1)

hit_data_square32 = []

for row in hit_data_padded:
    row_square = np.reshape(row, target_shape)
    img = Image.fromarray((row_square))
    img = img.resize((32,32), Image.ANTIALIAS)
    arr = np.array(img)
    hit_data_square32.append(arr)

In [ ]:
plt.imshow(hit_data_true_square[1])

In [ ]:
plt.imshow(hit_data_square[1])

In [ ]:
plt.imshow(hit_data_square32[1])

In [ ]:
#we ignore normalization for now

#split in train and test
from sklearn.model_selection import train_test_split
multiplexed_train, multiplexed_test, solution_train, solution_test = train_test_split(hit_data_square32, hit_true,test_size = 0.1)

In [ ]:
# define network

In [ ]:
n_classes = solution_train[0].shape[0]

In [ ]:
input_shape = (target_shape[0], target_shape[1], 1)

In [ ]:
n_classes

In [ ]:
np.array(hit_data_square32[0]).shape

In [11]:
model = tf.keras.applications.VGG16(
    include_top=True, weights=None,
    input_shape=(32,32,1),
    pooling=None, classes=305,
    classifier_activation='linear')

TypeError: ('Invalid keyword argument: %s', 'classifier_activation')

In [ ]:
model.summary()

In [ ]:
model.compile(loss="mean_absolute_error", optimizer='adam', metrics=['mae'])

In [ ]:
len(multiplexed_train)

In [ ]:
solution_train.shape

In [ ]:
type(multiplexed_train[0][0][0])

In [ ]:
np.expand_dims(multiplexed_train[0], axis=0)

In [ ]:
model.predict(np.expand_dims(multiplexed_train[0], axis=0))

In [ ]:
#do the training

history = model.fit(np.asarray(multiplexed_train), solution_train, validation_data = (np.asarray(multiplexed_test),solution_test), epochs=500)
# model.save('tests.h5')


In [ ]:
#setting up the NN

model = Sequential()
model.add(Dense(605, input_dim=305, activation='relu')) #61*5 input dimension
model.add(Dense(500, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(5, activation='linear')) #5 output dimension (up to hits)


In [ ]:
# import matplotlib.pyplot as plt
from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

solution_pred = model.predict(multiplexed_test)

fig=plt.figure()
i=0
diff = []

while i < len(solution_pred):
    diff.append(solution_test[i][0] - solution_pred[i][0])
    #diff.append(solution_test[i][1] - solution_pred[i][1])
    i+=1
diff = np.asarray(diff)
diff = diff[diff < 2]
diff = diff[diff > -2]
diff = diff.tolist()

(mu, sigma) = norm.fit(diff)
print(mu)
print(sigma)
binwidth = 0.1
n, bins, patches = plt.hist(diff, normed=1, bins=np.arange(min(diff), max(diff) + binwidth, binwidth))
y = mlab.normpdf(bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

plt.xlabel('Difference in Strips')
plt.xlim([-5,5])
plt.ylabel('Counts')
plt.show()